In [1]:
from sklearn import datasets
from scipy.io.arff import loadarff
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score

import numpy as np
import pandas as pd


/home/yannick/bin/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
forest = datasets.fetch_covtype()

In [3]:
type(forest)

sklearn.datasets.base.Bunch

In [4]:
X, X_test, y, y_test = train_test_split(
    forest.data, forest.target == 2, test_size=0.3, random_state=1234)

In [5]:
print(X.shape, y.shape, X_test.shape, y_test.shape)

(406708, 54) (406708,) (174304, 54) (174304,)


## Maintenant un peu de xgboost!

### D'abord un tout petit

In [6]:
import numpy as np
def logit(p):
    return np.log(p) - np.log(1 - p)

def inv_logit(p):
    return np.exp(p) / (1 + np.exp(p))

In [11]:
import xgboost as xgb

dtrain = xgb.DMatrix(X, label=y)
dtest  = xgb.DMatrix(X_test, label=y_test)
evallist = [(dtest, 'eval'), (dtrain, 'train')]

param = {
    'bst:max_depth': 5, 
    'bst:eta': 0.5, 
    'silent': True, 
    'verbose': 0,
    'objective': 'binary:logistic',
    'nthread': 4,
    'eval_metric': 'auc',
    'tree_method': 'gpu_hist'
}

num_round = 4
bst = xgb.train(param,
                dtrain,
                num_round,
                evallist,
                verbose_eval    = 40
               )


[0]	eval-auc:0.815255	train-auc:0.816074
[3]	eval-auc:0.866843	train-auc:0.867408


In [8]:
X_test[0, [0,25,35,15]]

array([ 2687.,     0.,     0.,     0.])

In [9]:
leaves = bst.predict(dtest, pred_leaf=True)
z = bst.predict(dtest)
leaves[:3], z[:3], inv_logit(0.388039+ 0.354413)

(array([[73, 77, 78, 81],
        [73, 78, 76, 88],
        [73, 77, 78, 82]], dtype=int32),
 array([ 0.75847626,  0.63132304,  0.7231403 ], dtype=float32),
 0.67753180825889836)

In [10]:
leaves[400], y[400], X_test[400, [0,7,3]]

(array([ 97, 102,  99,  89], dtype=int32),
 False,
 array([ 3126.,   254.,   283.]))

In [11]:
bst.dump_model("bst_2_trees")
! cat bst_2_trees

booster[0]:
0:[f0<3074] yes=1,no=2,missing=1
	1:[f0<2501] yes=3,no=4,missing=3
		3:[f0<2466] yes=7,no=8,missing=7
			7:[f44<0.5] yes=15,no=16,missing=15
				15:[f23<0.5] yes=31,no=32,missing=31
					31:[f46<0.5] yes=61,no=62,missing=61
						61:leaf=-0.587316
						62:leaf=0.381818
					32:[f12<0.5] yes=63,no=64,missing=63
						63:leaf=-0.537327
						64:leaf=-0.257143
				16:leaf=0.547826
			8:[f10<0.5] yes=17,no=18,missing=17
				17:[f23<0.5] yes=33,no=34,missing=33
					33:[f46<0.5] yes=65,no=66,missing=65
						65:leaf=-0.479532
						66:leaf=0.447273
					34:[f5<1358] yes=67,no=68,missing=67
						67:leaf=-0.0828358
						68:leaf=-0.573723
				18:[f9<5146] yes=35,no=36,missing=35
					35:[f5<760] yes=69,no=70,missing=69
						69:leaf=0.218182
						70:leaf=0.58209
					36:[f1<155] yes=71,no=72,missing=71
						71:leaf=-0.546667
						72:leaf=-0.0666667
		4:[f25<0.5] yes=9,no=10,missing=9
			9:[f35<0.5] yes=19,no=20,missing=19
				19:[f15<0.5] yes=37,no=38,missing=37
					37:[f36<

In [12]:
X.shape, y.shape, X_test.shape, y_test.shape

((406708, 54), (406708,), (174304, 54), (174304,))

In [16]:
%%time

import xgboost as xgb

dtrain = xgb.DMatrix(X, label=y)
dtest  = xgb.DMatrix(X_test, label=y_test)
evallist = [(dtest, 'eval'), (dtrain, 'train')]

param = {
    'bst:max_depth': 5, 
    'bst:eta': 0.5, 
    'silent': True, 
    'verbose': 0,
    'objective': 'binary:logistic',
    'nthread': 4,
    'eval_metric': 'auc',
    'tree_method': 'gpu_hist'
}

num_round = 601
bst = xgb.train(param,
                dtrain,
                num_round,
                evallist,
                verbose_eval    = 40
               )


[0]	eval-auc:0.815255	train-auc:0.816074
[40]	eval-auc:0.922714	train-auc:0.925346
[80]	eval-auc:0.944923	train-auc:0.948678
[120]	eval-auc:0.957583	train-auc:0.96204
[160]	eval-auc:0.964993	train-auc:0.969911
[200]	eval-auc:0.97121	train-auc:0.976483
[240]	eval-auc:0.975295	train-auc:0.980906
[280]	eval-auc:0.97817	train-auc:0.983926
[320]	eval-auc:0.980859	train-auc:0.986674
[360]	eval-auc:0.982833	train-auc:0.98864
[400]	eval-auc:0.98469	train-auc:0.990505
[440]	eval-auc:0.985995	train-auc:0.991836
[480]	eval-auc:0.987254	train-auc:0.993098
[520]	eval-auc:0.988187	train-auc:0.993982
[560]	eval-auc:0.989098	train-auc:0.994851
[600]	eval-auc:0.98963	train-auc:0.995409
CPU times: user 2min 51s, sys: 5.6 s, total: 2min 56s
Wall time: 44.4 s


In [17]:
%%time

import xgboost as xgb

dtrain = xgb.DMatrix(X, label=y)
dtest  = xgb.DMatrix(X_test, label=y_test)
evallist = [(dtest, 'eval'), (dtrain, 'train')]

param = {
    'bst:max_depth': 5, 
    'bst:eta': 0.5, 
    'silent': True, 
    'verbose': 0,
    'objective': 'binary:logistic',
    'nthread': 4,
    'eval_metric': 'auc',
    'tree_method': 'hist'
}

num_round = 601
bst = xgb.train(param,
                dtrain,
                num_round,
                evallist,
                verbose_eval    = 40
               )


[0]	eval-auc:0.815251	train-auc:0.816075
[40]	eval-auc:0.9265	train-auc:0.929326
[80]	eval-auc:0.94302	train-auc:0.946318
[120]	eval-auc:0.956107	train-auc:0.960191
[160]	eval-auc:0.964489	train-auc:0.969099
[200]	eval-auc:0.970554	train-auc:0.975652
[240]	eval-auc:0.975686	train-auc:0.98084
[280]	eval-auc:0.978581	train-auc:0.983946
[320]	eval-auc:0.981304	train-auc:0.98677
[360]	eval-auc:0.983529	train-auc:0.989086
[400]	eval-auc:0.984916	train-auc:0.990508
[440]	eval-auc:0.986242	train-auc:0.991837
[480]	eval-auc:0.987173	train-auc:0.992835
[520]	eval-auc:0.988132	train-auc:0.993809
[560]	eval-auc:0.988981	train-auc:0.99464
[600]	eval-auc:0.989828	train-auc:0.995467
CPU times: user 3min 23s, sys: 1.36 s, total: 3min 24s
Wall time: 51.3 s


In [17]:
%%time 

param = {
    'bst:max_depth': 7, 
    'bst:eta': 0.15,
    'colsample_bytree': 0.9,
    'subsample': 0.8,
    'silent': True, 
    'verbose': 0,
    'objective': 'binary:logistic',
    'nthread': 8,
    'eval_metric': 'auc',
    'tree_method': 'gpu_hist'
}

num_round = 1001
bst = xgb.train(param,
                dtrain,
                num_round,
                evallist,
                verbose_eval    = 200
               )


[0]	eval-auc:0.812394	train-auc:0.811757
[200]	eval-auc:0.971655	train-auc:0.976823
[400]	eval-auc:0.984686	train-auc:0.990596
[600]	eval-auc:0.98971	train-auc:0.995573
[800]	eval-auc:0.991967	train-auc:0.997739
[1000]	eval-auc:0.993169	train-auc:0.998751
CPU times: user 8min 37s, sys: 16.6 s, total: 8min 53s
Wall time: 1min 10s


In [13]:
%%time 

param = {
    'bst:max_depth': 7, 
    'bst:eta': 0.15,
    'colsample_bytree': 0.6,
    'subsample': 0.6,
    'silent': True, 
    'verbose': 0,
    'objective': 'binary:logistic',
    'nthread': 8,
    'eval_metric': 'auc'
}

num_round = 1001
bst = xgb.train(param,
                dtrain,
                num_round,
                evallist,
                verbose_eval    = 200
               )


[0]	eval-auc:0.813996	train-auc:0.814956
[200]	eval-auc:0.964482	train-auc:0.970541
[400]	eval-auc:0.979837	train-auc:0.987093
[600]	eval-auc:0.985612	train-auc:0.993219
[800]	eval-auc:0.988232	train-auc:0.996087
[1000]	eval-auc:0.989941	train-auc:0.997669
CPU times: user 25min 20s, sys: 11.9 s, total: 25min 32s
Wall time: 3min 20s


In [21]:
%%time

import json
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import mean_squared_error

# create dataset for lightgbm
lgb_train = lgb.Dataset(X, y)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# specify your configurations as a dict
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'auc'},
    'num_leaves': 31,
    'learning_rate': 0.2,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 40
}

print('Start training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=3000,
                valid_sets=lgb_eval,
                early_stopping_rounds=40,
                verbose_eval=200)

# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

print('Feature names:', gbm.feature_name())

# feature importances
print('Feature importances:', list(gbm.feature_importance()))

Start training...
Training until validation scores don't improve for 40 rounds.
[200]	valid_0's auc: 0.958904	valid_0's l2: 0.0851358
[400]	valid_0's auc: 0.975109	valid_0's l2: 0.0678659
[600]	valid_0's auc: 0.98177	valid_0's l2: 0.058823
[800]	valid_0's auc: 0.985688	valid_0's l2: 0.0528656
[1000]	valid_0's auc: 0.988335	valid_0's l2: 0.0481381
[1200]	valid_0's auc: 0.989881	valid_0's l2: 0.0450907
[1400]	valid_0's auc: 0.990939	valid_0's l2: 0.0427897
[1600]	valid_0's auc: 0.991836	valid_0's l2: 0.0407016
[1800]	valid_0's auc: 0.992415	valid_0's l2: 0.0392418
[2000]	valid_0's auc: 0.992905	valid_0's l2: 0.0379845
[2200]	valid_0's auc: 0.993308	valid_0's l2: 0.0368072
[2400]	valid_0's auc: 0.993671	valid_0's l2: 0.0357798
[2600]	valid_0's auc: 0.994003	valid_0's l2: 0.0348062
[2800]	valid_0's auc: 0.994297	valid_0's l2: 0.033995
[3000]	valid_0's auc: 0.994543	valid_0's l2: 0.0333161
Feature names: ['Column_0', 'Column_1', 'Column_2', 'Column_3', 'Column_4', 'Column_5', 'Column_6', 'C

In [14]:
%%time

import json
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import mean_squared_error

# create dataset for lightgbm
lgb_train = lgb.Dataset(X, y)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# specify your configurations as a dict
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'auc'},
    'num_leaves': 31,
    'learning_rate': 0.25,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0,
#    'device': 'gpu'
}

print('Start training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=1200,
                valid_sets=lgb_eval,
                early_stopping_rounds=5,
                verbose_eval=200)

# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

print('Feature names:', gbm.feature_name())

# feature importances
print('Feature importances:', list(gbm.feature_importance()))

Start training...
Training until validation scores don't improve for 5 rounds.
[200]	valid_0's auc: 0.962666	valid_0's l2: 0.0813967
[400]	valid_0's auc: 0.978688	valid_0's l2: 0.0631855
[600]	valid_0's auc: 0.984547	valid_0's l2: 0.0544109
[800]	valid_0's auc: 0.98774	valid_0's l2: 0.0487787
[1000]	valid_0's auc: 0.989613	valid_0's l2: 0.0450177
[1200]	valid_0's auc: 0.990915	valid_0's l2: 0.0422525
Feature names: ['Column_0', 'Column_1', 'Column_2', 'Column_3', 'Column_4', 'Column_5', 'Column_6', 'Column_7', 'Column_8', 'Column_9', 'Column_10', 'Column_11', 'Column_12', 'Column_13', 'Column_14', 'Column_15', 'Column_16', 'Column_17', 'Column_18', 'Column_19', 'Column_20', 'Column_21', 'Column_22', 'Column_23', 'Column_24', 'Column_25', 'Column_26', 'Column_27', 'Column_28', 'Column_29', 'Column_30', 'Column_31', 'Column_32', 'Column_33', 'Column_34', 'Column_35', 'Column_36', 'Column_37', 'Column_38', 'Column_39', 'Column_40', 'Column_41', 'Column_42', 'Column_43', 'Column_44', 'Colu

In [19]:
from catboost import CatBoostClassifier

clf = CatBoostClassifier(learning_rate=0.25,
                         depth=7,
                         rsm=0.9,
                         iterations=400, 
                         random_seed=42, eval_metric="AUC")

In [20]:
%%time 

clf.fit(X, y, eval_set=(X_test, y_test), verbose=True, plot=False)

Borders for float features generated
0:	learn 0.8029511072	test 0.8036969587	bestTest 0.8036969587		total: 411ms	remaining: 2m 44s
1:	learn 0.8163563314	test 0.8168610415	bestTest 0.8168610415		total: 827ms	remaining: 2m 44s
2:	learn 0.8304290198	test 0.8304887835	bestTest 0.8304887835		total: 1.23s	remaining: 2m 42s
3:	learn 0.8382811244	test 0.8383920514	bestTest 0.8383920514		total: 1.65s	remaining: 2m 43s
4:	learn 0.8462336886	test 0.8462224762	bestTest 0.8462224762		total: 2.1s	remaining: 2m 45s
5:	learn 0.8514833629	test 0.8513095764	bestTest 0.8513095764		total: 2.55s	remaining: 2m 47s
6:	learn 0.8546205459	test 0.8544506461	bestTest 0.8544506461		total: 3s	remaining: 2m 48s
7:	learn 0.8594201056	test 0.8594768173	bestTest 0.8594768173		total: 3.43s	remaining: 2m 48s
8:	learn 0.8635836369	test 0.8635318823	bestTest 0.8635318823		total: 3.86s	remaining: 2m 47s
9:	learn 0.8663623289	test 0.8663635647	bestTest 0.8663635647		total: 4.29s	remaining: 2m 47s
10:	learn 0.8693658895	test

87:	learn 0.9372399084	test 0.9357296093	bestTest 0.9357296093		total: 39.3s	remaining: 2m 19s
88:	learn 0.9375189092	test 0.9360299775	bestTest 0.9360299775		total: 39.7s	remaining: 2m 18s
89:	learn 0.9383095803	test 0.9368375768	bestTest 0.9368375768		total: 40.2s	remaining: 2m 18s
90:	learn 0.9388816659	test 0.937411524	bestTest 0.937411524		total: 40.6s	remaining: 2m 17s
91:	learn 0.9393661741	test 0.93789902	bestTest 0.93789902		total: 41.1s	remaining: 2m 17s
92:	learn 0.9396435828	test 0.9381700914	bestTest 0.9381700914		total: 41.6s	remaining: 2m 17s
93:	learn 0.9398202301	test 0.9383412254	bestTest 0.9383412254		total: 42s	remaining: 2m 16s
94:	learn 0.9401347363	test 0.9386298459	bestTest 0.9386298459		total: 42.5s	remaining: 2m 16s
95:	learn 0.9405737971	test 0.9390794937	bestTest 0.9390794937		total: 43s	remaining: 2m 16s
96:	learn 0.9408617444	test 0.9393289532	bestTest 0.9393289532		total: 43.4s	remaining: 2m 15s
97:	learn 0.9412075614	test 0.9396833538	bestTest 0.93968335

174:	learn 0.9591226962	test 0.9568691381	bestTest 0.9568691381		total: 1m 18s	remaining: 1m 41s
175:	learn 0.9592325343	test 0.9569662149	bestTest 0.9569662149		total: 1m 19s	remaining: 1m 40s
176:	learn 0.9594484663	test 0.9572016772	bestTest 0.9572016772		total: 1m 19s	remaining: 1m 40s
177:	learn 0.9596230007	test 0.9573484856	bestTest 0.9573484856		total: 1m 20s	remaining: 1m 39s
178:	learn 0.9597600372	test 0.9574758645	bestTest 0.9574758645		total: 1m 20s	remaining: 1m 39s
179:	learn 0.9599956283	test 0.9577016191	bestTest 0.9577016191		total: 1m 20s	remaining: 1m 38s
180:	learn 0.9601111993	test 0.9578222873	bestTest 0.9578222873		total: 1m 21s	remaining: 1m 38s
181:	learn 0.9602598023	test 0.957950902	bestTest 0.957950902		total: 1m 21s	remaining: 1m 37s
182:	learn 0.9604034756	test 0.9580795235	bestTest 0.9580795235		total: 1m 22s	remaining: 1m 37s
183:	learn 0.9606444409	test 0.9583023027	bestTest 0.9583023027		total: 1m 22s	remaining: 1m 37s
184:	learn 0.9608775401	test 0.9

259:	learn 0.9694175253	test 0.9666666229	bestTest 0.9666666229		total: 1m 56s	remaining: 1m 2s
260:	learn 0.9695234829	test 0.9667763087	bestTest 0.9667763087		total: 1m 56s	remaining: 1m 2s
261:	learn 0.9696124439	test 0.9668649862	bestTest 0.9668649862		total: 1m 57s	remaining: 1m 1s
262:	learn 0.9696906813	test 0.9669309612	bestTest 0.9669309612		total: 1m 57s	remaining: 1m 1s
263:	learn 0.9697677891	test 0.96699718	bestTest 0.96699718		total: 1m 58s	remaining: 1m
264:	learn 0.9698505658	test 0.9670762186	bestTest 0.9670762186		total: 1m 58s	remaining: 1m
265:	learn 0.9699324939	test 0.9671438391	bestTest 0.9671438391		total: 1m 59s	remaining: 60s
266:	learn 0.970018623	test 0.9671983795	bestTest 0.9671983795		total: 1m 59s	remaining: 59.5s
267:	learn 0.9700942957	test 0.9672670314	bestTest 0.9672670314		total: 1m 59s	remaining: 59s
268:	learn 0.9701735636	test 0.9673480476	bestTest 0.9673480476		total: 2m	remaining: 58.6s
269:	learn 0.9702190076	test 0.967398689	bestTest 0.9673986

346:	learn 0.97550313	test 0.972336171	bestTest 0.972336171		total: 2m 33s	remaining: 23.5s
347:	learn 0.9755445487	test 0.9723728993	bestTest 0.9723728993		total: 2m 34s	remaining: 23s
348:	learn 0.9755899074	test 0.9724194756	bestTest 0.9724194756		total: 2m 34s	remaining: 22.6s
349:	learn 0.9756519314	test 0.9724744056	bestTest 0.9724744056		total: 2m 35s	remaining: 22.2s
350:	learn 0.9757415795	test 0.9725647019	bestTest 0.9725647019		total: 2m 35s	remaining: 21.7s
351:	learn 0.9758369572	test 0.972656971	bestTest 0.972656971		total: 2m 35s	remaining: 21.3s
352:	learn 0.9759169717	test 0.9727338789	bestTest 0.9727338789		total: 2m 36s	remaining: 20.8s
353:	learn 0.9759749192	test 0.9727833413	bestTest 0.9727833413		total: 2m 36s	remaining: 20.4s
354:	learn 0.9760266716	test 0.9728414307	bestTest 0.9728414307		total: 2m 37s	remaining: 19.9s
355:	learn 0.9760937972	test 0.9729158473	bestTest 0.9729158473		total: 2m 37s	remaining: 19.5s
356:	learn 0.9761552364	test 0.9729792852	bestTe